Import the jar file manually

In [2]:
! sudo cp /home/jovyan/work/jars/neo4j-connector-apache-spark_2.12-4.1.0_for_spark_3.jar /usr/local/spark/jars/neo4j-connector-apache-spark_2.12-4.1.0_for_spark_3.jar

23/02/20 21:03:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Exception in thread "main" java.nio.file.NoSuchFileException: /tmp/tmp_vzxn6fy/connection3563081863024998923.info
	at java.base/sun.nio.fs.UnixException.translateToIOException(UnixException.java:92)
	at java.base/sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:111)
	at java.base/sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:116)
	at java.base/sun.nio.fs.UnixFileSystemProvider.newByteChannel(UnixFileSystemProvider.java:219)
	at java.base/java.nio.file.Files.newByteChannel(Files.java:371)
	at java.base/java.nio.file.Files.createFile(Files.java:648)
	at java.base/java.nio.file.TempFileHelper.create(TempFile

In [3]:
import pyspark
from pyspark.sql import SparkSession

# NEO4J  CONFIGURATION
bolt_url = "bolt://neo4j:7687"
# Spark init
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print(bolt_url)


23/02/20 21:04:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


bolt://neo4j:7687


5. Load a Spark DataFrame of USA country suppliers and their products for products that are not discontinued. Include supplier company and contact names, country, and phone. From products, include product name, discontinued, unit price, and units in stock.

In [4]:
cypher_query = '''
MATCH (s:Supplier)-[:SUPPLIES]->(p:Product)
WHERE s.country = 'USA' AND p.discontinued = false
RETURN s.companyName, s.contactName, s.country, p.productName,
p.discontinued, p.unitPrice, p.unitsInStock
'''

df = spark.read.format("org.neo4j.spark.DataSource")\
    .option("url", bolt_url)\
    .option("query", cypher_query)\
    .load()
print(bolt_url)

bolt://neo4j:7687


In [5]:
df.printSchema()

root
 |-- s.companyName: string (nullable = true)
 |-- s.contactName: string (nullable = true)
 |-- s.country: string (nullable = true)
 |-- p.productName: string (nullable = true)
 |-- p.discontinued: boolean (nullable = true)
 |-- p.unitPrice: double (nullable = true)
 |-- p.unitsInStock: long (nullable = true)



In [6]:
df.toPandas()

,s.companyName,s.contactName,s.country,p.productName,p.discontinued,p.unitPrice,p.unitsInStock
0,New Orleans Cajun Delights,Shelley Burke,USA,Chef Anton's Cajun Seasoning,False,22.00,53
1,New Orleans Cajun Delights,Shelley Burke,USA,Louisiana Fiery Hot Pepper Sauce,False,21.05,76
2,New Orleans Cajun Delights,Shelley Burke,USA,Louisiana Hot Spiced Okra,False,17.00,4
3,Grandma Kelly's Homestead,Regina Murphy,USA,Uncle Bob's Organic Dried Pears,False,30.00,15
4,Grandma Kelly's Homestead,Regina Murphy,USA,Northwoods Cranberry Sauce,False,40.00,6
5,Grandma Kelly's Homestead,Regina Murphy,USA,Grandma's Boysenberry Spread,False,25.00,120
6,Bigfoot Breweries,Cheryl Saylor,USA,Sasquatch Ale,False,14.00,111
7,Bigfoot Breweries,Cheryl Saylor,USA,Steeleye Stout,False,18.00,20
8,Bigfoot Breweries,Cheryl Saylor,USA,Laughing Lumberjack Lager,False,14.00,22
9,New England Seafood Cannery,Robb Merchant,USA,Jack's New England Clam Chowder,False,9.65,85


6. Load the /datasets/fudgemart/fudgemart-employees.json into a Spark DataFrame. Make sure to create a column employee_name, which combines the first and last names together

In [7]:
file_name = "file:///home/jovyan/datasets/fudgemart/fudgemart-employees.json"
employees = spark.read.option("multiline", True).json(file_name)
employees.toPandas()
print(file_name)

,employee_birthdate,employee_department,employee_firstname,employee_fulltime,employee_hiredate,employee_hourlywage,employee_id,employee_jobtitle,employee_lastname,employee_ssn,employee_supervisor_id,employee_termdate
0,1982-01-12T00:00:00,Electronics,Arial,0,2011-07-05T00:00:00,15.2830,1,Sales Associate,Photo,111220001,5.0,None
1,1982-11-30T00:00:00,Electronics,Sal,1,2005-07-26T00:00:00,15.0699,2,Sales Associate,Ladd,111220002,5.0,None
2,1972-09-03T00:00:00,Hardware,Dustin,0,2004-07-02T00:00:00,12.4500,3,Sales Associate,Dawind,111220003,6.0,2010-11-06T00:00:00
3,1990-05-13T00:00:00,Hardware,Sandi,1,2011-01-02T00:00:00,13.0997,4,Sales Associate,Shores,111220004,6.0,None
4,1974-02-22T00:00:00,Electronics,Isabelle,1,2005-08-16T00:00:00,20.1287,5,Department Manager,Gunnering,111220005,32.0,None
5,1973-07-29T00:00:00,Hardware,Lee,1,2004-01-26T00:00:00,18.5845,6,Department Manager,Hvmeehom,111220006,32.0,None
6,1988-04-03T00:00:00,Housewares,Allan,1,2005-02-22T00:00:00,12.0878,7,Sales Associate,Wrench,111220007,24.0,None
7,1973-12-13T00:00:00,Sporting Goods,Ally,1,2005-08-14T00:00:00,12.3541,8,Sales Associate,Gator,111220008,27.0,None
8,1980-11-30T00:00:00,Housewares,Alma,1,2004-11-12T00:00:00,11.7152,9,Sales Associate,Frienzergon,111220009,24.0,None
9,1976-10-02T00:00:00,Hardware,Artie,1,2005-04-04T00:00:00,12.7269,10,Sales Associate,Choke,111220010,6.0,None


In [8]:
employees.printSchema()

root
 |-- employee_birthdate: string (nullable = true)
 |-- employee_department: string (nullable = true)
 |-- employee_firstname: string (nullable = true)
 |-- employee_fulltime: string (nullable = true)
 |-- employee_hiredate: string (nullable = true)
 |-- employee_hourlywage: double (nullable = true)
 |-- employee_id: long (nullable = true)
 |-- employee_jobtitle: string (nullable = true)
 |-- employee_lastname: string (nullable = true)
 |-- employee_ssn: string (nullable = true)
 |-- employee_supervisor_id: long (nullable = true)
 |-- employee_termdate: string (nullable = true)



In [23]:
from pyspark.sql.functions import col, concat, lit
employees = spark.read.option("multiline", True).json(file_name)\
    .withColumn("employee_name", concat(col("employee_firstname"),lit(" "), col("employee_lastname")))
#employees.printSchema()
tmp = employees.select("employee_id", "employee_name", "employee_jobtitle", "employee_department")
print(file_name)
tmp.toPandas()

file:///home/jovyan/datasets/fudgemart/fudgemart-employees.json


,employee_id,employee_name,employee_jobtitle,employee_department
0,1,Arial Photo,Sales Associate,Electronics
1,2,Sal Ladd,Sales Associate,Electronics
2,3,Dustin Dawind,Sales Associate,Hardware
3,4,Sandi Shores,Sales Associate,Hardware
4,5,Isabelle Gunnering,Department Manager,Electronics
5,6,Lee Hvmeehom,Department Manager,Hardware
6,7,Allan Wrench,Sales Associate,Housewares
7,8,Ally Gator,Sales Associate,Sporting Goods
8,9,Alma Frienzergon,Sales Associate,Housewares
9,10,Artie Choke,Sales Associate,Hardware


7. In Spark, load the employees into Neo4j under the label node Employee; include employee_name, employee_department, employee_id, and employee_jobtitle as node attributes. Make sure  employee_name is the first attribute as this will be the node’s visible label. Provide evidence the nodes were created in Neo4i UI with a Cypher query. 

In [34]:
cypher_merge = '''
MERGE (e:Employee 
    { 
        dept: event.employee_department,
        name: event.employee_name,
        jobtitle: event.employee_jobtitle, 
        id: event.employee_id 
    })
'''

tmp.write.format("org.neo4j.spark.DataSource").mode("Overwrite")\
    .option("url", bolt_url)\
    .option("query", cypher_merge)\
    .save()
print(cypher_merge)


MERGE (e:Employee 
    { 
        dept: event.employee_department,
        name: event.employee_name,
        jobtitle: event.employee_jobtitle, 
        id: event.employee_id 
    })


MERGE (e:Employee 
    { 
        dept: event.employee_department,
        name: event.employee_name,
        jobtitle: event.employee_jobtitle, 
        id: event.employee_id 
    })



In [40]:
cypher_query = '''
MATCH (e:Employee)
RETURN e.name, e.dept, e.jobtitle, e.id;
'''

df = spark.read.format("org.neo4j.spark.DataSource")\
    .option("url", bolt_url)\
    .option("query", cypher_query)\
    .load()
df.show()
print(bolt_url)

+------------------+--------------+------------------+----+
|            e.name|        e.dept|        e.jobtitle|e.id|
+------------------+--------------+------------------+----+
|       Arial Photo|   Electronics|   Sales Associate|   1|
|          Sal Ladd|   Electronics|   Sales Associate|   2|
|     Dustin Dawind|      Hardware|   Sales Associate|   3|
|      Sandi Shores|      Hardware|   Sales Associate|   4|
|Isabelle Gunnering|   Electronics|Department Manager|   5|
|      Lee Hvmeehom|      Hardware|Department Manager|   6|
|      Allan Wrench|    Housewares|   Sales Associate|   7|
|        Ally Gator|Sporting Goods|   Sales Associate|   8|
|  Alma Frienzergon|    Housewares|   Sales Associate|   9|
|       Artie Choke|      Hardware|   Sales Associate|  10|
|       Bette Alott|Sporting Goods|   Sales Associate|  11|
|      Bill Melator|Sporting Goods|   Sales Associate|  12|
|       Bob Enweave|Sporting Goods|   Sales Associate|  13|
|   Chris P. Nugget|   Electronics|   Sa

8. In Spark add a SUPERVISES relationship to the nodes. Basically, you must match two nodes—one where the ID is the employee_id and the other where the ID is the employee_supervisor_id—and then merge a relationship.

In [44]:
cipher_sql = '''
MATCH(e:Employee)
MATCH(s:Employee)
WHERE e.id = event.employee_id and s.id = event.employee_supervisor_id
MERGE (s)-[:SUPERVISES]->(e)
'''

sup_employee = employees.select("employee_supervisor_id", "employee_id").orderBy("employee_supervisor_id")
sup_employee.toPandas()

sup_employee.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", bolt_url)\
    .option("query", cipher_sql)\
    .save()
print(cipher_sql)



MATCH(e:Employee)
MATCH(s:Employee)
WHERE e.id = event.employee_id and s.id = event.employee_supervisor_id
MERGE (s)-[:SUPERVISES]->(e)

